In [3]:
import cv2
import dlib
import numpy as np
import time
import os
from skimage.transform import resize

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
FACIAL_LANDMARKS_IDXS = dict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 35)),
    ("jaw", (0, 17))
])

#VIDEO_FPS = 25
#VIDEO_FRAMES_PER_WORD = 30

MOUTH_SHAPE_FROM = FACIAL_LANDMARKS_IDXS["mouth"][0]
MOUTH_SHAPE_TO = FACIAL_LANDMARKS_IDXS["mouth"][1]

In [5]:
def make_rect_shape_square(rect):
    # Rect: (x, y, w, h)
    # If width > height
    if rect[2] > rect[3]:
        rect = (rect[0], int(rect[1] + rect[3] / 2 - rect[2] / 2),
                rect[2], rect[2])
    # Else (height > width)
    else:
        rect = (int(rect[0] + rect[2] / 2 - rect[3] / 2), rect[1],
                rect[3], rect[3])
    # Return
    return rect


def expand_rect(rect, scale=1.5, frame_shape=(256, 256)):
    # Rect: (x, y, w, h)
    w = int(rect[2] * scale)
    h = int(rect[3] * scale)
    x = max(0, min(frame_shape[1] - w, rect[0] - int((w - rect[2]) / 2)))
    y = max(0, min(frame_shape[0] - h, rect[1] - int((h - rect[3]) / 2)))
    return (x, y, w, h)


In [22]:
def convert_to_images(path,label,basename,p):
    #win = dlib.image_window()
    #path_load = os.path.join(path)
    cap = cv2.VideoCapture(path)
    #base = os.path.basename(path)
    path_save = os.path.join('save/',label)
    path_save = os.path.join(path_save,p)
    path_save = os.path.join(path_save,basename)
    folder = os.path.exists(path_save)
    if not folder:
        os.makedirs(path_save)
        print("--- new folder ---")
    
    #cap = cv2.VideoCapture('data/ABOUT/train/ABOUT_00001.mp4')
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print(video_length)
    #path_save='/Users/wubingzhang/Desktop/mnist/save/ABOUT/'
    count=0
    while cap.isOpened():
    
        ret, cv_img = cap.read()
        if cv_img is None:
            break

        # OpenCV默认是读取为RGB图像，而dlib需要的是BGR图像，因此这一步转换不能少
        img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)# 检测人脸
        faces = detector(img, 1)
        #print(type(dets))
        if len(faces) == 0:
            img_blank = np.zeros((120, 120, 3), np.uint8)
            print("no face")

        if len(faces) > 1:
            for face in faces:

            # Predict facial landmarks
                shape = predictor(cv_img, face)

                # Note all mouth landmark coordinates
                mouthCoords = np.array([[shape.part(i).x, shape.part(i).y]
                                        for i in range(MOUTH_SHAPE_FROM, MOUTH_SHAPE_TO)])

                # Check if correct face is selected by checking position of mouth mean
                mouthMean = np.mean(mouthCoords, axis=0)
                if mouthMean[0] > 110 and mouthMean[0] < 150 \
                        and mouthMean[1] > 140 and mouthMean[1] < 170:
                    break
            print("more than 1 face")
            
            mouthRect = (np.min(mouthCoords[:, 0]), np.min(mouthCoords[:, 1]),
                     np.max(mouthCoords[:, 0]) - np.min(mouthCoords[:, 0]),
                     np.max(mouthCoords[:, 1]) - np.min(mouthCoords[:, 1]))

            mouthRect = make_rect_shape_square(mouthRect)

        # Expand mouthRect square
            expandedMouthRect = expand_rect(mouthRect,
                scale=(0.5 * face.width() / mouthRect[2]),
                frame_shape=(cv_img.shape[0], cv_img.shape[1]))
            
            img_blank = np.round(resize(cv_img[expandedMouthRect[1]:expandedMouthRect[1] + expandedMouthRect[3],
                                                  expandedMouthRect[0]:expandedMouthRect[0] + expandedMouthRect[2]],
                                            (120, 120), preserve_range=True)).astype('uint8')
            

        if len(faces) == 1:
            face = faces[0]
            # Predict facial landmarks
            shape = predictor(cv_img, face)
            # Note all mouth landmark coordinates
            mouthCoords = np.array([[shape.part(i).x, shape.part(i).y]
                                    for i in range(MOUTH_SHAPE_FROM, MOUTH_SHAPE_TO)])
            
            mouthRect = (np.min(mouthCoords[:, 0]), np.min(mouthCoords[:, 1]),
                     np.max(mouthCoords[:, 0]) - np.min(mouthCoords[:, 0]),
                     np.max(mouthCoords[:, 1]) - np.min(mouthCoords[:, 1]))

            mouthRect = make_rect_shape_square(mouthRect)

        # Expand mouthRect square
            expandedMouthRect = expand_rect(mouthRect,
                scale=(0.5 * face.width() / mouthRect[2]),
                frame_shape=(cv_img.shape[0], cv_img.shape[1]))
            
            img_blank = np.round(resize(cv_img[expandedMouthRect[1]:expandedMouthRect[1] + expandedMouthRect[3],
                                                  expandedMouthRect[0]:expandedMouthRect[0] + expandedMouthRect[2]],
                                            (120, 120), preserve_range=True)).astype('uint8')
        #for k, d in enumerate(dets):
            # Mouth Rect: x, y, w, h
#         mouthRect = (np.min(mouthCoords[:, 0]), np.min(mouthCoords[:, 1]),
#                      np.max(mouthCoords[:, 0]) - np.min(mouthCoords[:, 0]),
#                      np.max(mouthCoords[:, 1]) - np.min(mouthCoords[:, 1]))

#         mouthRect = make_rect_shape_square(mouthRect)

#         # Expand mouthRect square
#         expandedMouthRect = expand_rect(mouthRect,
#             scale=(0.5 * face.width() / mouthRect[2]),
#             frame_shape=(cv_img.shape[0], cv_img.shape[1]))

        # Resize to 120x120
#         img_blank = np.round(resize(cv_img[expandedMouthRect[1]:expandedMouthRect[1] + expandedMouthRect[3],
#                                                   expandedMouthRect[0]:expandedMouthRect[0] + expandedMouthRect[2]],
#                                             (120, 120), preserve_range=True)).astype('uint8')

        count=count+1
    #             #print(count)
    #             img_blank = cv2.cvtColor(img_blank, cv2.COLOR_BGR2RGB)
    #             #cv2.imshow("face_"+str(count), img_blank)

    #           # 保存人脸到本地
        cv2.imwrite(path_save+"/%#05d.jpg" % (count), img_blank)


    cap.release()

In [23]:
part = ['test','train','val']

In [24]:
path=os.path.join("data","ABSOLUTELY")
path=os.path.join(path,"train")
path=os.path.join(path,"ABSOLUTELY_00903.mp4")
convert_to_images(path,"AAAAA","aaa","XXX")

28
no face
no face


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [25]:
labels = os.listdir("data/")
for i, label in enumerate(labels):
    if label != '.DS_Store':
        path=os.path.join("data", label)
        for p in part:
            #print(i,label)
            path_part=os.path.join(path, p)
            for file in os.listdir(path_part):
                if file.endswith(".mp4"):
                    #print(file)
                    base = os.path.splitext(file)[0]
                    path_file=os.path.join(path_part, file)
                    #path='data/ABOUT/train/'
                    print(path_file)
                    convert_to_images(path_file,label,base,p)

data/ABSOLUTELY/test/ABSOLUTELY_00048.mp4
28


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


data/ABSOLUTELY/test/ABSOLUTELY_00049.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00017.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00003.mp4
28
no face
data/ABSOLUTELY/test/ABSOLUTELY_00002.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00016.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00014.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00028.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00029.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00015.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00001.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00039.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00005.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00011.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00010.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00004.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00038.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00012.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00006.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00007.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00013.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00036.mp4
28
data/ABSOLUTELY/test/ABSOLUTELY_00022.mp4
28
no

data/ABSOLUTELY/train/ABSOLUTELY_00112.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00674.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00660.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00106.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00099.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00927.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00933.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00728.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00066.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00700.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00714.mp4
28
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
more than 1 face
data/ABSOLUTELY/train/ABSOLUTELY_00072.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00258.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00502.mp4
28
data/ABSOLUTELY/

data/ABSOLUTELY/train/ABSOLUTELY_00160.mp4
28
more than 1 face
data/ABSOLUTELY/train/ABSOLUTELY_00148.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00941.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00799.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00955.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00766.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00772.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00014.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00028.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00564.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00202.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00216.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00570.mp4
28
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
data/ABSOLUTELY/train/ABSOLUTELY_00558.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00559.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00217.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00571.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00565.mp4
28
data/ABSOLUTELY/train/ABSOLUTELY_00203.mp4


...

data/ABUSE/train/ABUSE_00719.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00080.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00094.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00902.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00916.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00241.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00527.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00533.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00255.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00269.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00282.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00296.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00335.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00453.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00447.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00321.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00309.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00490.mp4
--- new folder ---
28
data/ABUSE

data/ABUSE/train/ABUSE_00327.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00333.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00455.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00482.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00496.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00119.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00125.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00643.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00657.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00131.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00864.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00870.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00680.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00858.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00694.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00855.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00841.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00699.mp4
--- new folder ---
28
data/ABUSE

data/ABUSE/train/ABUSE_00264.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00258.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00933.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00099.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00927.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00714.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00072.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00066.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00700.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00728.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00729.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00067.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00701.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00715.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00073.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00098.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00926.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00932.mp4
--- new folder ---
28
data/ABUSE

data/ABUSE/train/ABUSE_00176.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00638.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00823.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00837.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00189.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00821.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00835.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00809.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00148.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00606.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00160.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00174.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00612.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00389.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00438.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00362.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00404.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00410.mp4
--- new folder ---
28
data/ABUSE

data/ABUSE/train/ABUSE_00012.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00984.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00990.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00748.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00749.mp4
--- new folder ---
28
more than 1 face
more than 1 face
more than 1 face
more than 1 face
data/ABUSE/train/ABUSE_00991.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00985.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00775.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00013.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00007.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00761.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00952.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00946.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00239.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00211.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00577.mp4
--- new folder ---
28
data/ABUSE/train/ABUSE_00563.mp4
--- new folder ---


In [23]:
cap = cv2.VideoCapture('data/ABSOLUTELY/test/ABSOLUTELY_00002.mp4')
count=0
path_save = os.path.join('save/','try')

while cap.isOpened():
    
    ret, cv_img = cap.read()
    if cv_img is None:
        break

    # OpenCV默认是读取为RGB图像，而dlib需要的是BGR图像，因此这一步转换不能少
    img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)# 检测人脸
    faces = detector(img, 1)
    #print(type(dets))
    if len(faces) == 0:
        print("no face")
        
    if len(faces) > 1:
        print("more than 1 face")
        
    if len(faces) == 1:
        face = faces[0]
        # Predict facial landmarks
        shape = predictor(cv_img, face)
        # Note all mouth landmark coordinates
        mouthCoords = np.array([[shape.part(i).x, shape.part(i).y]
                                for i in range(MOUTH_SHAPE_FROM, MOUTH_SHAPE_TO)])
    #for k, d in enumerate(dets):
        # Mouth Rect: x, y, w, h
    mouthRect = (np.min(mouthCoords[:, 0]), np.min(mouthCoords[:, 1]),
                 np.max(mouthCoords[:, 0]) - np.min(mouthCoords[:, 0]),
                 np.max(mouthCoords[:, 1]) - np.min(mouthCoords[:, 1]))
    
    mouthRect = make_rect_shape_square(mouthRect)

    # Expand mouthRect square
    expandedMouthRect = expand_rect(mouthRect,
        scale=(0.5 * face.width() / mouthRect[2]),
        frame_shape=(cv_img.shape[0], cv_img.shape[1]))

    # Resize to 120x120
    img_blank = np.round(resize(cv_img[expandedMouthRect[1]:expandedMouthRect[1] + expandedMouthRect[3],
                                              expandedMouthRect[0]:expandedMouthRect[0] + expandedMouthRect[2]],
                                        (120, 120), preserve_range=True)).astype('uint8')

    count=count+1
#             #print(count)
#             img_blank = cv2.cvtColor(img_blank, cv2.COLOR_BGR2RGB)
#             #cv2.imshow("face_"+str(count), img_blank)

#           # 保存人脸到本地
    cv2.imwrite(path_save+"/%#05d.jpg" % (count+1), img_blank)


cap.release()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
